# Análise exploratória dos Dados

## Sobre o Dataset

Este é um dataset sobre pedidos em ecommerce brasileiro realizado pela Olist Store. O dataset contém informações sobre 100 mil pedidos feitos em diversas lojas durante o período de 2016 a 2018. O dataset permite visualizar os pedidos a partir de múltiplas dimensões. De status do pedido, preço, pagamento e performance do frete até a localização do cliente, atributos do produto e, finalmente, avaliações escritas pelos clientes. Também há um conjunto de dados de geolocalização que relaciona os códigos postais brasileiros às coordenadas lat/long.

São dados comerciais reais, foram anonimizados e as referências às empresas e parceiros no texto da revisão foram substituídas pelos nomes das grandes casas de Game of Thrones.


## Atenção
1. Um pedido pode ter vários itens.
2. Cada item pode ser atendido por um vendedor distinto.
3. Todo o texto identificando lojas e parceiros foi substituído pelos nomes das grandes casas de Game of Thrones.

## Data Schema
Os dados são divididos em vários conjuntos de dados para melhor compreensão e organização. Consulte o esquema de dados a seguir ao trabalhar com ele:

<img src="assets/data_schema.png" width="1000px" />



---

**Customers Dataset (dataset de clientes)**<br/><br/>
Este conjunto de dados contém informações sobre o cliente e sua localização. Use-o para identificar clientes únicos no conjunto de dados de pedidos e para encontrar o local de entrega dos pedidos.

No sistema, cada pedido é atribuído a um customerid exclusivo. Isso significa que o mesmo cliente receberá IDs diferentes para pedidos diferentes. O objetivo de ter um customerunique_id no conjunto de dados é permitir que você identifique clientes que fizeram recompras na loja. Caso contrário, você descobriria que cada pedido tinha um cliente diferente associado.

---

**Geolocation Dataset (Conjunto de dados de geolocalização)**<br/><br/>
Este conjunto de dados contém informações sobre códigos postais brasileiros e suas coordenadas lat/lng. Use-o para traçar mapas e encontrar distâncias entre vendedores e clientes.

---

**Order Items Dataset (Conjunto de dados de itens de pedido)**<br/><br/>
Este conjunto de dados inclui dados sobre os itens comprados em cada pedido.

---

**Payments Dataset (Conjunto de dados de pagamentos)**<br/><br/>
Este conjunto de dados inclui dados sobre as opções de pagamento dos pedidos.

---

**Order Reviews Dataset (Conjunto de dados de avaliações de pedidos)**<br/><br/>
Este conjunto de dados inclui dados sobre as avaliações feitas pelos clientes.

Depois que um cliente compra o produto da Olist Store, um vendedor é notificado para atender a esse pedido. Assim que o cliente recebe o produto, ou a data prevista de entrega está prevista, o cliente recebe um inquérito de satisfação por e-mail onde pode dar nota da experiência de compra e deixar alguns comentários.

---

**Order Dataset (Conjunto de dados do pedido)**<br/><br/>
Este é o conjunto de dados principal. De cada pedido você pode encontrar todas as outras informações.

---

**Products Dataset (Conjunto de dados de produtos)**<br/><br/>
Este conjunto de dados inclui dados sobre os produtos vendidos pela Olist.

---

**Sellers Dataset (Conjunto de dados de vendedores)**<br/><br/>
Este conjunto de dados inclui dados sobre os vendedores que atenderam aos pedidos feitos na Olist. Use-o para encontrar a localização do vendedor e identificar qual vendedor entregou cada produto.

---

**Tradução do nome da categoria**<br/><br/>
Traduz o nome da categoria do produto para o inglês.

---

# Análise Exploratória por Datasets

## 1) Orders_dataset (Dataset de Pedidos)

**Ideias:**
* Calcular a quantidade de tempo desde o momento da compra até a entrega no cliente

In [23]:
# Análise dos dados
import pandas as pd
import numpy as np

# Visualização dos dados
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go

# Bibliotecas de utilidades
from include.graficos import exibirGrafico

import warnings
warnings.filterwarnings("ignore")

In [24]:
dfOrders = pd.read_csv('data/orders_dataset.csv')

dfCustomers = pd.read_csv('data/customers_dataset.csv')

In [25]:
print(dfOrders.shape)

dfOrders.head(5)

(99441, 8)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [26]:
dfOrders.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [27]:
dfOrders.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [28]:
dfOrders = pd.merge(dfOrders, dfCustomers, left_on='customer_id', right_on='customer_id', how='left')

del dfCustomers

In [29]:
dfOrders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,barreiras,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP


In [30]:
dfOrders.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'customer_unique_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state'],
      dtype='object')

In [31]:
dfOrders['order_purchase_timestamp'] = pd.to_datetime(dfOrders['order_purchase_timestamp'])
dfOrders['order_approved_at'] = pd.to_datetime(dfOrders['order_approved_at'])
dfOrders['order_delivered_carrier_date'] = pd.to_datetime(dfOrders['order_delivered_carrier_date'])
dfOrders['order_delivered_customer_date'] = pd.to_datetime(dfOrders['order_delivered_customer_date'])
dfOrders['order_estimated_delivery_date'] = pd.to_datetime(dfOrders['order_estimated_delivery_date'])

In [32]:
dfOrders.dtypes

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
customer_unique_id                       object
customer_zip_code_prefix                  int64
customer_city                            object
customer_state                           object
dtype: object

In [33]:
def convertToTimesOfDay(value):
    result = ''
    hour = value.hour

    if hour >= 5 and hour < 12:
        result = 'manha'
    elif hour >= 12 and hour < 19:
        result = 'tarde'
    elif hour >= 19 and hour <= 23:
        result = 'noite'
    else:
        result = 'madrugada'

    return result


In [34]:
dfOrders['times_of_day'] = dfOrders['order_purchase_timestamp'].apply(convertToTimesOfDay)

dfOrders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,times_of_day
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,manha
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,noite
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,manha
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,noite
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,noite


In [35]:
dfOrders['times_of_day'].value_counts()

tarde        44130
noite        28331
manha        22428
madrugada     4552
Name: times_of_day, dtype: int64

In [36]:
dfOrders['order_date_purchase'] = pd.to_datetime(dfOrders['order_purchase_timestamp']).dt.date

dfOrders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,times_of_day,order_date_purchase
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,manha,2017-10-02
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,noite,2018-07-24
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,manha,2018-08-08
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,noite,2017-11-18
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,noite,2018-02-13


---

### 1.) Análise de Série Temporal de Pedidos

In [49]:
tsPurchase = dfOrders['order_date_purchase'].value_counts().reset_index().sort_values(by=['index'])

tsPurchase.columns = ['date', 'count']
tsPurchase['date'] = pd.to_datetime(tsPurchase['date'])


print(tsPurchase.shape)
tsPurchase.head()

(634, 2)


,date,count
627,2016-09-04,1
632,2016-09-05,1
629,2016-09-13,1
626,2016-09-15,1
617,2016-10-02,1


In [51]:
fig = exibirGrafico(tsPurchase, x='date', y='count', type='line', title = 'Pedidos durante todo o período', \
                    width = 1000, height = 400, xlabel = 'Período', ylabel = 'Quantidade de Pedidos')

fig.show()

Observações:
* Vemos que nas pontas dos gráficos há poucos registros. Provavelmente se trata de uma pequena amostra que não representa de fato o que aconteceu neste dataset (isto é uma suposição). Portanto, como analista tomo a decisão de remover os dados anteriores ao dia `05 jan de 2017` e dados superiores ao dia `20 de ago de 2018`.

In [52]:
tsPurchase = tsPurchase.loc[(tsPurchase['date'] > '2017-01-04') & (tsPurchase['date'] < '2018-08-21')]

print(tsPurchase.shape)
tsPurchase.head()

(593, 2)


,date,count
590,2017-01-05,32
608,2017-01-06,4
611,2017-01-07,4
606,2017-01-08,6
607,2017-01-09,5


In [53]:
fig = exibirGrafico(tsPurchase, x='date', y='count', type='line', title = 'Pedidos durante todo o período', \
                    width = 1000, height = 400, xlabel = 'Período', ylabel = 'Quantidade de Pedidos')

fig.show()

In [68]:
tsCount = tsPurchase.sort_values(by='count', ascending=False)[0:10]
tsCount.sort_values(by=['date'], inplace=True)
tsCount['date'] = tsCount['date'].dt.strftime('%b %d, %Y')

fig = exibirGrafico(tsCount, x='date', y='count', type='bar', title = 'Top 10 Dias com mais vendas', \
                    width = 1000, height = 400, xlabel = 'Data', ylabel = 'Quantidade de Pedidos')

fig.show()

---

## 2) Orders_items (Dataset de Itens de Pedidos)

**Ideias:**


In [ ]:
dfItems = pd.read_csv('data/order_items_dataset.csv')

In [ ]:
dfItems.head(5)

In [ ]:
dfItems.shape

---

## 3) Order_payments (Tipos de Pagamentos de Pedidos)

**Ideias:**

In [ ]:
dfPayments = pd.read_csv('data/order_payments_dataset.csv')

dfPayments.head()

In [ ]:
dfPayments['payment_type'].value_counts()

---

## 4) Orders_reviews (Avaliações de Pedidos)

**Ideias:**

In [ ]:
dfReviews = pd.read_csv('data/order_reviews_dataset.csv')

dfReviews.head()

In [ ]:
dfReviews['review_score'].value_counts()

---

## 5) Products_dataset (Dataset de Produtos)

**Ideias**

In [ ]:
dfProducts = pd.read_csv('data/products_dataset.csv')

In [ ]:
print(dfProducts.shape)
display(dfProducts.head())

In [ ]:
dfProducts = dfProducts[['product_id', 'product_category_name']]

dfProducts.head()

---

## 6) Customers_dataset (Dataset dos compradores)

**Ideias:**

In [ ]:
dfCustomers = pd.read_csv('data/customers_dataset.csv')

print(dfCustomers.shape)

dfCustomers.head()

## 7) Sellers_dataset (Dataset de vendedores)

**Ideias:**

In [ ]:
dfSellers = pd.read_csv('data/sellers_dataset.csv')

print(dfSellers.shape)

dfSellers.head()

---

## 8) Geolocation_dataset (Dataset de Geolocalização)

**Ideias:**

In [ ]:
dfGeolocation = pd.read_csv('data/geolocation_dataset.csv')

dfGeolocation.head()

In [ ]:
dfGeolocation.shape